# Populating Spatial Coverage for DCAT Data Portals

## Part 1: Introduction
This Jupyter Notebook is intended to find the spatial coverage based on bounding box for DCAT Data Portals. It is a reverse version of **<a href='https://github.com/BTAA-Geospatial-Data-Project/geonames'>geonames</a>**.

## Part 2: Preparation
We will be using **Jupyter Notebook(anaconda 3)** to edit and run the script. Information on Anaconda installation can be found <a href='https://docs.anaconda.com/anaconda/install/'>here</a>. Please note that this script is running on Python 3.

Before running the script, you may need to:
### 1. Inspect the csv file
Check if it includes all the required columns with proper name
- required columns
    1. Download
    2. Slug
    3. Publisher
    4. Code
    5. Bounding Box
    
### 2. Run other Jupyter Notebooks
- If the target state(s) hasn't been converted into city or county bounding box file, you may need to 
    1. download county and city boundary file (GeoJSON or Shapefile) online
    2. run `city_boundary.ipynb` or `county_boundary.ipynb` to create boundary GeoJSON files 
        - if there exists regional data portals, you may need to run `merge_geojson.ipynb` to merge them together
    3. run `city_bbox.ipynb` or `county_bbox.ipynb` to create bounding box GeoJSON files

### 3. Restructure Directories
- `sjoin.ipynb`
- **geojsonScripts** folder
    - `city_boundary.ipynb`
    - `county_boundary.ipynb`
    - `city_bbox.ipynb`
    - `county_bbox.ipynb`
    - `merge_geojsons.ipynb`
- **geojsons** folder
    - **State** foloder
        - *State_County_boundaries.json*
        - *State_City_boundaries.json*
        - *State_County_bbox.json*
        - *State_City_bbox.json*
    - **portalCode** foloder (Multiple states)
        - *portalCode_County_boundaries.json*
        - *portalCode_City_boundaries.json*
        - *portalCode_County_bbox.json*
        - *portalCode_City_bbox.json*
    - ...
- **reports** folder
    - one csv file formatted in GBL Metadata Template
        
The final product would be one CSV file. 

> Original created on Feb 4 2021 <br>
> @author: Yijing Zhou @YijingZhou33

## Part 3: Get Started

### Step 1: Import modules

In [ ]:
import pandas as pd
import os
import geopandas as gpd
import json
from shapely.geometry import box
import numpy as np
from itertools import chain
from itertools import repeat
from functools import reduce
import time
import urllib.request
import requests

### Step 2: Set file path

In [ ]:
# ActionDate = time.strftime('%Y%m%d')
# newItemscsv = os.path.join('reports', f'allNewItems_{ActionDate}.csv')

##### CSV file with metadata #####
csvname = 'imagery'
newItemscsv = os.path.join('reports', csvname + '.csv')
df_csv = pd.read_csv(newItemscsv, encoding = 'unicode_escape')

### Step 3: Check if link is valid

In [ ]:
print("\n--------------------- Check if link is valid: --------------------\n")

def check_url(df, timeout):
    totalcount = len(df.index)
    countnotshp = countprivate = countok = count404 = count500 = countothercode = countconnection = counttimeout = 0
    start_time = time.time()
    filesize = download = imageserver = None
    filesizelist, downloadlist, imageserverlist, oklist, checkagainlist = ([] for i in range(5))
    for _, row in df.iterrows():
        slug = row['Slug']
        ## access the download link
        if row['Format'] == 'Imagery':
            url = row['ImageServer']
        else:
            url = row['Download']
        try:
            ## set timeout to avoid waiting for the server to response forever             
            response = requests.get(url, timeout = timeout, proxies = urllib.request.getproxies())
            response.raise_for_status()
            ## vector data: check if it is a shapefile  
            ## only keep the data source url
            if 'content-type' in response.headers and response.headers['content-type'] == 'application/json; charset=utf-8':
                countnotshp += 1
                oklist.append(slug)
                print(f'{slug}: Not a shapefile')
            ## imagery Data: check if we could access ImageServer
            ## only keep the data source url
            elif 'Cache-Control' in response.headers and response.headers['Cache-Control'] == 'private':
                countprivate += 1
                oklist.append(slug)
                print(f'{slug}: Could not access ImageServer')  
            else:
                ## if records with both vaild data source page and download link
                ## query the file size and keep both links
                if 'content-length' in response.headers:
                    filesize = str(round(int(response.headers['content-length']) / 1000000, 4)) + ' MB'
                if row['Download'] is not None:
                    download = row['Download']
                if row['ImageServer'] is not None:
                    imageserver = row['ImageServer']
                countok += 1
                oklist.append(slug)
                print(f'{slug}: Success')
        ## check HTTP error: 404 (not found) or 500 (server error)       
        except requests.exceptions.HTTPError as errh:
            ## 404 error: drop this record
            if errh.response.status_code == 404:
                count404 += 1
            ## 500 error: only keeps data source url    
            elif errh.response.status_code == 500:
                count500 += 1
                oklist.append(slug)
            ## other HTTP errors: only keeps data source url
            else:
                countothercode += 1
            print (f'{slug}: {errh}')
        ## check Connection error: need to be double-checked by increasing the timeout or even manually open it    
        except requests.exceptions.ConnectionError as errc:
            download = row['Download']
            imageserver = row['ImageServer']
            countconnection += 1
            checkagainlist.append(slug)
            print (f'{slug}: {errc}')
        ## check Timeout error: need to be double-checked by increasing the timeout or even manually open it 
        except requests.exceptions.Timeout as errt:
            download = row['Download']
            imageserver = row['ImageServer']            
            counttimeout += 1
            checkagainlist.append(slug)
            print (f'{slug}: {errt}')
        
        filesizelist.append(filesize)
        downloadlist.append(download)
        imageserverlist.append(imageserver)
    
    df['FileSize'] = filesizelist    
    df['Download'] = downloadlist 
    df['ImageServer'] = imageserverlist 
    
    errordict = {'OK': countok, 'Not a shapefile': countnotshp, 'Timeout Error': counttimeout,
                 'Could not access ImageServer': countprivate, '404 Not Found': count404, 
                 '500 Internal Server Error': count500, 'Other HTTP Errors': countothercode, 
                 'Connection Errors': countconnection}     
    msglist = [f'{k}: {v}, {round(v/totalcount * 100.0, 2)}%' for k, v in errordict.items()]
    print('\n---------- %s seconds ----------' % round((time.time() - start_time), 0), 
          '\n\n---------- Error Summary ----------', 
          '\nAll records: %s' % totalcount)
    for msg in msglist:
        print(msg)
    
    ## records with runtime error need to be double-checked
    return [df[df['Slug'].isin(oklist)], df[df['Slug'].isin(checkagainlist)]]

df_total = check_url(df_csv, 3)
df_ok = df_total[0].reset_index(drop = True)
df_checkagain = df_total[1].reset_index(drop = True)

In [ ]:
## set the timeout as 10 seconds
## if there still exists any records, manually check the download link to see if it works
if len(df_checkagain.index):
    df_checkagain = check_url(df_checkagain, 10)
    df_checkok = df_checkagain[0].reset_index(drop = True)
    df_manualcheck = df_checkagain[1].reset_index(drop = True)
    df_manualcheck['Title'] = 'Manually check this link!'
    df_csv = pd.concat([df_ok, df_checkok, df_manualcheck]).reset_index(drop = True)

### Step 4: Split csv file if necessary

In [ ]:
## if records come from Esri, the spatial coverage is considered as United States
df_esri = df_csv[df_csv['Publisher'] == 'Esri'].reset_index(drop=True)
df_csv = df_csv[df_csv['Publisher'] != 'Esri'].reset_index(drop=True)

### Step 5: Splite state from column 'Publisher'
The portal code is the main indicator: <br>
- 01 - Indiana
- 02 - Illinois
- 03 - Iowa
- 04 - Maryland
- 04c-01 - District of Columbia
- 04f-01 - Delaware, Philadelphia, Maryland, New Jersey
- 05 - Minnesota
- 06 - Michigan
- 07 - Michigan
- 08 - Pennsylvania
- 09 - Indiana
- 10 - Wisconsin
- 11 - Ohio
- 12 - Illinois
- 13 - Nebraska
- 99 - Esri

In [ ]:
print("\n--------------------- Populating spatial coverage based on bounding box: --------------------\n")

statedict = {'01': 'Indiana', '02': 'Illinois', '03': 'Iowa', '04': 'Maryland', '04c-01': 'District of Columbia', 
             '04f-01': '04f-01', '05': 'Minnesota', '06': 'Michigan', '07': 'Michigan', '08': 'Pennsylvania', 
             '09': 'Indiana', '10': 'Wisconsin', '11': 'Ohio', '12': 'Illinois', '13': 'Nebraska', '99': 'Esri'}

df_csv['State'] = [statedict[row['Code']] if row['Code'] in statedict.keys(
                    ) else statedict[row['Code'][0:2]] for _, row in df_csv.iterrows()]

## Part 4: Build up GeoJSON dataframe

### Step 6: Create bounding box for csv file

In [ ]:
def format_coordinates(df, identifier):
    ## create regular bouding box coordinate pairs and round them to 2 decimal places
    ## manually generates the buffering zone
    df = pd.concat([df, df['Bounding Box'].str.split(',', expand=True).astype(float).round(2)], axis=1).rename(
        columns={0:'minX', 1:'minY', 2:'maxX', 3:'maxY'})
    
    ## check if there exists wrong coordinates and drop them
    coordslist = ['minX', 'minY', 'maxX', 'maxY']
    idlist = []
    for _, row in df.iterrows():
        for coord in coordslist:
            ## e.g. [-180.0000,-90.0000,180.0000,90.0000]             
            if abs(row[coord]) == 0 or abs(row[coord]) == 180:
                idlist.append(row[identifier])
        if (row.maxX - row.minX) > 10 or (row.maxY - row.minY) > 10:
            idlist.append(row[identifier])
    
    ## create bounding box
    df['Coordinates'] = df.apply(lambda row: box(row.minX, row.minY, row.maxX, row.maxY), axis=1)
    df['Roundcoords'] = df.apply(lambda row: ', '.join([str(i) for i in [row.minX, row.minY, row.maxX, row.maxY]]), axis=1)
    
    ## clean up unnecessary columns
    df = df.drop(columns = coordslist).reset_index(drop = True)
    
    df_clean = df[~df[identifier].isin(idlist)]
    ## remove records with wrong coordinates into a new dataframe
    df_wrongcoords = df[df[identifier].isin(idlist)].drop(columns = ['State', 'Coordinates'])
    
    return [df_clean, df_wrongcoords]

df_csvlist = format_coordinates(df_csv, 'Slug')
df_clean = df_csvlist[0]
df_wrongcoords = df_csvlist[1]

### Step 7: Convert csv and GeoJSON file into dataframe

In [ ]:
gdf_rawdata = gpd.GeoDataFrame(df_clean, geometry = df_clean['Coordinates'])
gdf_rawdata.crs = 'EPSG:4326'

### Step 8: Split dataframe and convert them into dictionary 

In [ ]:
## e.g.
## splitdict = {'Minnesota': {'Roundcoords 1': df_1, 'Roundcoords 2': df_2}, 
##              'Michigan':  {'Roundcoords 3': df_3, ...}, 
##               ...}

splitdict = {}
for state in list(gdf_rawdata['State'].unique()):
    gdf_slice = gdf_rawdata[gdf_rawdata['State'] == state]
    if state:
        coordsdict = {}
        for coord in list(gdf_slice['Roundcoords'].unique()):
            coordsdict[coord] = gdf_slice[gdf_slice['Roundcoords'] == coord].drop(columns = ['State', 'Roundcoords'])
        splitdict[state] = coordsdict
    else:
        sluglist = gdf_slice['Code'].unique()
        print("Can't find the bounding box file: ", sluglist)

## Part 5: Spatial Join
**<a href='https://geopandas.org/reference/geopandas.sjoin.html#geopandas-sjoin'>`geopandas.sjoin`</a>** provides the following the criteria used to match rows:
- intersects 
- within
- contains

### Step 9: Perform spatial join on each record

In [ ]:
def split_placename(df, level):
    formatlist = []
    for _, row in df.iterrows():
        ## e.g. 'Baltimore County, Baltimore City'
        ## --> ['Baltimore County&Maryland', 'Baltimore City&Maryland']
        if row[level] != 'nan':
            placelist = row[level].split(', ')
            formatname = ', '.join([(i + '&' + row['State']) for i in placelist])  
        ## e.g. 'nan'
        ## --> ['nan']
        else:
            formatname = 'nan'
        formatlist.append(formatname)
    return formatlist

""" Spatial Join: both city and county bounding box files """
def city_and_county_sjoin(gdf_rawdata, op, state):
    bboxpath = os.path.join('geojsons', state, f'{state}_City_bbox.json')
    gdf_basemap = gpd.read_file(bboxpath)
    ## spatial join
    df_merged = gpd.sjoin(gdf_rawdata, gdf_basemap, op = op, how = 'left')[['City', 'County', 'State']].astype(str)
    # merge column 'City', 'County' into one column 'op'
    df_merged['City'] = split_placename(df_merged, 'City')
    df_merged['County'] = split_placename(df_merged, 'County')
    df_merged[op] = df_merged[['City', 'County']].agg(', '.join, axis=1).replace('nan, nan', 'nan')
    # convert placename into list
    oplist = df_merged[op].astype(str).values.tolist()
    return oplist

""" Spatial Join: either city or county bounding box file """
def city_or_county_sjoin(gdf_rawdata, op, state, level):
    bboxpath = os.path.join('geojsons', state, f'{state}_{level}_bbox.json')
    gdf_basemap = gpd.read_file(bboxpath)
    ## spatial join
    df_merged = gpd.sjoin(gdf_rawdata, gdf_basemap, op = op, how = 'left')[[level, 'State']].astype(str)
    # merge column level and 'State' into one column 'op'
    df_merged[op] = df_merged.apply(lambda row: (row[level] + '&' + row['State']) if str(row[level]) != 'nan' else 'nan', axis = 1)
    # convert placename into list
    oplist = df_merged[op].astype(str).values.tolist()
    return oplist

### Step 10: Remove duplicates and 'nan' from place name

In [ ]:
def remove_nan(row):
    ## e.g. ['nan', 'Minneapolis, Minnesota', 'Hennepin County, Minnesota', 'Hennepin County, Minnesota']
    ## remove 'nan' and duplicates from list: ['Minneapolis, Minnesota, 'Hennepin County, Minnesota']
    nonan = list(filter(lambda x: x != 'nan', row))
    nodups = list(set(', '.join(nonan).split(', ')))
    result = [i.replace('&', ', ') for i in nodups]
    return result

### Step 11: Fetch the proper join bouding box files

In [ ]:
operations = ['intersects', 'within', 'contains']
def spatial_join(gdf_rawdata, state, length):
    oplist = []
    for op in operations:
        bboxpath = os.path.join('geojsons', state, f'{state}_City_bbox.json')
        
        ## Disteict of Columbia doesn't have county boudning box file
        if state == 'District of Columbia':
            columbia = city_or_county_sjoin(gdf_rawdata, op, state, 'City')
            pnamelist = remove_nan(columbia)
        
        ## check if there exists bounding box files
        elif os.path.isfile(bboxpath):
            city_county_state_list = city_and_county_sjoin(gdf_rawdata, op, state)
            county_state_list = city_or_county_sjoin(gdf_rawdata, op, state, 'County')
            pnamelist = city_county_state_list + county_state_list
            pnamelist = remove_nan(pnamelist)
       
        ## missing bounding box file    
        else: 
            print('Missing city bounding box file: ', state)
            continue 
        
        oplist.append(pnamelist)
    ## duplicate placename list for all rows with the same bounding box    
    allopslist = list(repeat(oplist, length))
    ## ultimately it returns a dataframe with placename related to matching operation
    ## e.g. dataframe = {'intersects', 'within', 'contains'}
    df_match = pd.DataFrame.from_records(allopslist, columns = operations)
    return df_match

### Step 12: Merge place names generated by three matching operations to raw data

In [ ]:
mergeddf = []
## loop through splitdict based on key 'State'
for state, gdfdict in splitdict.items():
    ## loop through records based on key 'Bounding Box'
    for coord, gdf_split in gdfdict.items():
        length = int(len(gdf_split))
        df_comparison = spatial_join(gdf_split.iloc[[0]], state, length)
        ## append dataframe {'intersects', 'within', 'contains'} to raw data
        gdf_sjoin = pd.concat([gdf_split.reset_index(drop=True), df_comparison.reset_index(drop=True)], axis = 1)
        mergeddf.append(gdf_sjoin)

## merge all dataframes with different bounding boxes into one
gdf_merged = reduce(lambda left, right: left.append(right), mergeddf).reset_index(drop = True)

## replace [''] with None
for op in operations:
    gdf_merged[op] = gdf_merged[op].apply(lambda row: None if row == [''] else row)

## Part 6: Populate place names

### Step 13: Format spatial coverage based on GBL Metadata Template

In [ ]:
## e.g. ['Camden County, New Jersey', 'Delaware County, Pennsylvania', 'Philadelphia County, Pennsylvania']
def format_placename(colname):
    inv_map = {}
    plist = []

    ## {'Camden County': 'New Jersey', 'Delaware County': 'Pennsylvania', 'Philadelphia County': 'Pennsylvania'}
    namedict = dict(item.split(', ') for item in colname)

    ## {'New Jersey': ['Camden County'], 'Pennsylvania': ['Delaware County', 'Philadelphia County']}
    for k, v in namedict.items():
        inv_map[v] = inv_map.get(v, []) + [k] 
    
    ## ['Camden County, New Jersey|New Jersey', 'Delaware County, Pennsylvania|Philadelphia County, Pennsylvania|Pennsylvania']
    for k, v in inv_map.items():
        pname = [elem + ', ' + k for elem in v]
        pname.append(k)
        plist.append('|'.join(pname))

    ## Camden County, New Jersey|New Jersey|Delaware County, Pennsylvania|Philadelphia County, Pennsylvania|Pennsylvania
    return '|'.join(plist)

### Step 14: Select spatial coverage based on operaions

In [ ]:
def populate_placename(df, identifier):
    placenamelist = []
    for _, row in df.iterrows():
        if row['contains'] is None:
            if row['intersects'] is None: 
                placename = ''
            elif row['within'] is None:
                placename = format_placename(row['intersects']) 
            else: 
                placename = format_placename(row['within']) 
        else:
            placename = format_placename(row['contains']) 
        placenamelist.append(placename)
    df['Spatial Coverage'] = placenamelist
    return df.drop(columns = ['intersects', 'within', 'contains', 'Coordinates', 'geometry'])

df_bbox = populate_placename(gdf_merged, 'Slug')

## Part 7: Write to csv file

In [ ]:
## check if there exists data portal from Esri
if len(df_esri):
    df_esri['Spatial Coverage'] = 'United States'
    
dflist = [df_esri, df_bbox, df_wrongcoords]
df_final = pd.concat(filter(len, dflist), ignore_index=True)

df_final.to_csv(newItemscsv, index = False)

print("\n--------------------- Congrats! ╰(￣▽￣)╯ --------------------\n")